\# Домашнее задание: Pydantic


## Важно!

- При выполнении задания используем точные типы (`EmailStr`, `HttpUrl`, `SecretStr`, `Decimal`, конкретные `Enum`).
- Придерживаемся принципа разделения валидаций: проверка поля — в `field_validator`, сквозные зависимости — в `model_validator`


## Задача 1. Профиль пользователя (валидация полей)

Постройте модель профиля пользователя для внутренней CRM:

**Требования**
1. Обязательные поля: `id: UUID`, `email: EmailStr`, `name: str`.
2. Опциональные поля: `website: HttpUrl | None`, `bio: str | None`.
3. Пароль хранится как `SecretStr`, должен быть не короче 8 символов.
4. Имя (`name`) нормализуйте: тримминг + одна пробельная последовательность между словами + первая буква каждого слова заглавная.
5. Если указан `website`, домен сайта не должен совпадать с доменом `email` (смысл: личный сайт != корпоративная почта).

Подсказки: используйте `field_validator` для нормализации и локальных проверок; и `model_validator(mode="after")` для проверки зависимости `email` ↔ `website`.


In [6]:
!pip install -U "pydantic[email,timezone]" -q

In [10]:
from typing import Optional
from pydantic import BaseModel, Field, EmailStr, HttpUrl, SecretStr
from pydantic import field_validator, model_validator
from uuid import UUID

class UserProfile(BaseModel):
    """Модель пользователя"""
    
    id: UUID
    email: EmailStr
    name: str
    password: SecretStr
        
    website: Optional[HttpUrl] = None
    bio: Optional[str] = None

    @field_validator("name")
    @classmethod
    def normalize_name(cls, v: str) -> str:
        parts = v.split()
        if not parts:
            raise ValueError("Поле не должно быть пустым.")
        return " ".join(word.capitalize() for word in parts)

    @field_validator("password")
    @classmethod
    def password_strength(cls, v: SecretStr) -> SecretStr:
        raw = v.get_secret_value()
        if len(raw) < 8:
            raise ValueError("Длина пароля должна быть не менее 8 симоволов.")
        return v

    @model_validator(mode="after")
    def check_domains(self):
        if self.website is None:
            return self

        email_domain = self.email.split("@")[-1].lower()

        website_str = str(self.website)
        host_part = website_str.split("://", 1)[-1]
        host_part = host_part.split("/", 1)[0]
        website_domain = host_part.split(":", 1)[0].lower()

        if email_domain == website_domain:
            raise ValueError("Домен сайта не должен совпадать с доменом почты.")

        return self

## Задача 2. Валидация функции заказа (`@validate_call`)

Реализуйте функцию `place_order`, которая принимает:
- `user_id: UUID`
- `sku: str` (артикул, только заглавные буквы/цифры, длина 3–12)
- `quantity: int` (>0)
- `price: Decimal` (>= 0), округляется банковским методом до 2 знаков

Функция должна возвращать словарь с ключами: `user_id`, `sku`, `quantity`, `price`, `amount` (quantity × price).

Используйте `@validate_call` и локальные проверки через обычный код (или вспомогательные валидаторы `TypeAdapter` не используем).


In [13]:
from pydantic import validate_call
from decimal import Decimal, ROUND_HALF_EVEN
from uuid import UUID
import re

# TODO: реализуйте функцию с @validate_call
@validate_call
def place_order(user_id: UUID, sku: str, quantity: int, price: Decimal):
    
    if not re.fullmatch(r"[A-Z0-9]{3,12}$", sku):
        raise ValueError("Артикул должен состоять только из заглавных букв и цифр, и иметь длину от 3 до 12 символов")

    if quantity <= 0:
        raise ValueError("Количество товара не  может быть меньше или равно 0")

    if price < 0:
        raise ValueError("Цена не может быть отрицательным значением")

    price = price.quantize(Decimal("0.01"), rounding=ROUND_HALF_EVEN)

    amount = (price * quantity).quantize(Decimal("0.01"), rounding=ROUND_HALF_EVEN)

    return {
        "user_id": user_id,
        "sku": sku,
        "quantity": quantity,
        "price": price,
        "amount": amount,
    }

## Задача 3. Модель заказа с бизнес-правилами

Смоделируйте заказ в магазине цифровых товаров.

**Требования**
- `OrderStatus: Enum` со значениями `new`, `paid`, `delivered`, `canceled`.
- Модель `OrderItem`:
  - `sku: str` как в задаче 2
  - `qty: int` (>0)
  - `unit_price: Decimal` (>=0) округление до 2 знаков
- Модель `Order`:
  - `id: UUID`
  - `user_email: EmailStr`
  - `items: list[OrderItem]` (не пустой)
  - `status: OrderStatus = 'new'`
  - `created_at: datetime` (по умолчанию `datetime.utcnow`)
  - Расчитанное поле `total: Decimal` — сумма по всем позициям
  - В `model_validator(mode="after")` запретите переход в `paid`/`delivered` при `total == 0` и запретите пустые корзины.

**Важно:** используйте только инструменты `pydantic` и стандартную библиотеку.


In [15]:
from pydantic import BaseModel, EmailStr, field_validator, model_validator
from typing import List
from decimal import Decimal, ROUND_HALF_EVEN
from uuid import UUID
from datetime import datetime
from enum import Enum

SKU_RE = re.compile(r"^[A-Z0-9]{3,12}$")

class OrderStatus(str, Enum):
    # TODO: перечислите статусы
    new = "new"
    paid = "paid"
    delivered = "delivered"
    canceled = "canceled"

class OrderItem(BaseModel):
    # TODO: опишите поля
    sku: str
    qty: int
    unit_price: Decimal

    @field_validator("sku")
    @classmethod
    def sku_format(cls, v: str) -> str:
        if not SKU_RE.fullmatch(v):
            raise ValueError("Артикул должен состоять только из заглавных букв и цифр, и иметь длину от 3 до 12 символов")
        return v

    @field_validator("qty")
    @classmethod
    def qty_positive(cls, v: int) -> int:
        if v <= 0:
            raise ValueError("Количество товара не  может быть меньше или равно 0")
        return v

    @field_validator("unit_price")
    @classmethod
    def price_non_negative(cls, v: Decimal) -> Decimal:
        if v < 0:
            raise ValueError("Цена не может быть отрицательным значением")
        return v.quantize(Decimal("0.01"), rounding=ROUND_HALF_EVEN)

class Order(BaseModel):
    # TODO: опишите поля
    id: UUID
    user_email: EmailStr
    items: List[OrderItem]
    status: OrderStatus = OrderStatus.new
    created_at: datetime = datetime.utcnow()
    total: Decimal = Decimal("0")

    @model_validator(mode="after")
    def check_business_rules(self):
        if not self.items:
            raise ValueError("Корзина не должгна быть пустой")

        total = Decimal("0")
        for item in self.items:
            line = (item.unit_price * item.qty).quantize(
                Decimal("0.01"), rounding=ROUND_HALF_EVEN
            )
            total += line

        self.total = total.quantize(Decimal("0.01"), rounding=ROUND_HALF_EVEN)

        if self.total == 0 and self.status in {OrderStatus.paid, OrderStatus.delivered}:
            raise ValueError('Статус заказа не может быть "оплачен" или "доставлен", если сумма заказа равна 0')

        return self

## Задача 4. Конфигурация приложения (`BaseSettings`)

Опишите настройки подключения к внешнему API:

- `APISettings(BaseSettings)` с полями:
  - `base_url: HttpUrl`
  - `token: SecretStr`
  - `timeout_sec: int = 5` (1–60)
  - `retries: int = 2` (0–10)
- Используйте `model_config = ConfigDict(env_prefix="API_", env_file=".env", extra="ignore")`
- Проверьте, что значения корректно читаются из переменных окружения.

В тесте ниже среда заполняется вручную.


In [17]:
!pip install -q pydantic-settings

In [20]:
import os
from pydantic_settings import BaseSettings
from pydantic import ConfigDict, SecretStr, HttpUrl, field_validator

class APISettings(BaseSettings):
    base_url: HttpUrl
    token: SecretStr
    timeout_sec: int = 5
    retries: int = 2

    @field_validator("timeout_sec", "retries")
    @classmethod
    def check_ranges(cls, v: int, info):
        field = info.field_name

        if field == "timeout_sec":
            if not (1 <= v <= 60):
                raise ValueError("timeout_sec должен быть в диапазоне от 1 до 60 секунд")
        elif field == "retries":
            if not (0 <= v <= 10):
                raise ValueError("retries должен быть в диапазоне от 0 до 10 попыток")

        return v

    model_config = ConfigDict(
        env_prefix="API_",
        env_file=".env",
        extra="ignore",
    )

## Задача 5. Извлечение из ORM (`from_attributes=True`)

Создайте простую SQLAlchemy-модель `SAUser(id, email, is_active)` (in-memory, без БД) и соответствующую модель Pydantic:

- Pydantic-модель `UserOut` с полями `id: UUID`, `email: EmailStr`, `is_active: bool`.
- Включите поддержку `from_attributes` в `model_config`.
- Создайте инстанс `SAUser` и провалидируйте его через `UserOut.model_validate(sa_user_instance)`.

Проверьте, что преобразование сработало.


In [22]:
from typing import Optional
from sqlalchemy import Column, String, Boolean
from sqlalchemy.orm import declarative_base
from uuid import uuid4, UUID
from pydantic import BaseModel, EmailStr, ConfigDict

Base = declarative_base()


class SAUser(Base):
    __tablename__ = "users"

    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    email = Column(String, nullable=False)
    is_active = Column(Boolean, default=True)

    def __init__(self, email: str, is_active: bool = True):
        self.id = str(uuid4())
        self.email = email
        self.is_active = is_active

class UserOut(BaseModel):
    id: UUID
    email: EmailStr
    is_active: bool

    model_config = ConfigDict(
        from_attributes=True
    )

**Проверка**

In [23]:
sa_user = SAUser(email="test@example.com", is_active=True)

user_out = UserOut.model_validate(sa_user)

print("SQLAlchemy id:", sa_user.id, type(sa_user.id))
print("Pydantic id:", user_out.id, type(user_out.id))
print(user_out)

SQLAlchemy id: 54240f6f-147d-4aa7-9bb9-592b6f3bc6e5 <class 'str'>
Pydantic id: 54240f6f-147d-4aa7-9bb9-592b6f3bc6e5 <class 'uuid.UUID'>
id=UUID('54240f6f-147d-4aa7-9bb9-592b6f3bc6e5') email='test@example.com' is_active=True


## Задача 6. JSON Schema и дружелюбные ошибки

1. Для модели из задачи 3 сгенерируйте JSON Schema (метод `model_json_schema`) и запишите его в переменную `ORDER_SCHEMA`.
2. Реализуйте функцию `safe_create_order(data: dict) -> tuple[bool, str]`, которая:
   - пытается создать `Order` из входного `dict`,
   - при успехе возвращает `(True, "<total=...>")`,
   - при ошибке возвращает `(False, "<короткое сообщение об ошибке>")` без стек-трейса.

Не используйте сторонние библиотеки.


In [26]:
import json

ORDER_SCHEMA = Order.model_json_schema()

print(json.dumps(ORDER_SCHEMA, indent=2, ensure_ascii=False))

def safe_create_order(data: dict) -> tuple[bool, str]:
    # TODO: реализуйте безопасное создание заказа
    try:
        order = Order.model_validate(data)
        return True, f"total={order.total}"
    except Exception as exc:
        return False, str(exc)

{
  "$defs": {
    "OrderItem": {
      "properties": {
        "sku": {
          "title": "Sku",
          "type": "string"
        },
        "qty": {
          "title": "Qty",
          "type": "integer"
        },
        "unit_price": {
          "anyOf": [
            {
              "type": "number"
            },
            {
              "pattern": "^(?!^[-+.]*$)[+-]?0*\\d*\\.?\\d*$",
              "type": "string"
            }
          ],
          "title": "Unit Price"
        }
      },
      "required": [
        "sku",
        "qty",
        "unit_price"
      ],
      "title": "OrderItem",
      "type": "object"
    },
    "OrderStatus": {
      "enum": [
        "new",
        "paid",
        "delivered",
        "canceled"
      ],
      "title": "OrderStatus",
      "type": "string"
    }
  },
  "properties": {
    "id": {
      "format": "uuid",
      "title": "Id",
      "type": "string"
    },
    "user_email": {
      "format": "email",
      "title": "User E